## VQE

In [ ]:
from src.modes import EstimatorMode, RunConfig, ExpectationEngine
from src.hamiltonians import build_sparse_hamiltonian
from src.ansatz import build_hva_layers
from src.initparams import find_best_initial_params, find_best_initial_params_counts
from src.energy import compute_exact_ground_energy
from src.optimize import optimize_energy
from qiskit_ibm_runtime.fake_provider import FakeQuitoV2

# --- Mode wählen ---
# 1) STATEVECTOR ohne Messungen:
# run_cfg = RunConfig(mode=EstimatorMode.STATEVECTOR, backend=None)

# 2) QASM ohne Noise (ideal):
# run_cfg = RunConfig(mode=EstimatorMode.QASM_IDEAL, backend=FakeQuitoV2())

# 3) QASM mit Fake-Noise:
run_cfg = RunConfig(mode=EstimatorMode.QASM_FAKE, backend=FakeQuitoV2(), shots=1024)

engine = ExpectationEngine(run_cfg)

# Problem
hx, hz, J = 0.10526315789473673, 2.0, -1
H, pauli_strings = build_sparse_hamiltonian(hx=hx, hz=hz, backend=run_cfg.backend, J=J, return_paulis=True)

# Initialguess – wähle je nach Pfad:
if run_cfg.mode == EstimatorMode.STATEVECTOR:
    x0, E_exact = find_best_initial_params(H, run_cfg.backend, num_layers=1)
else:
    x0, E_exact = find_best_initial_params_counts(H, pauli_strings, run_cfg.backend, num_layers=1)

# Cost-Funktion auf den Switch normiert
def cost(params):
    circ = build_hva_layers(params, backend=run_cfg.backend, num_layers=1)
    E = engine.energy(H, circ)
    return abs(E - E_exact) / abs(E_exact)

x_opt, _ = optimize_energy(x0, H, run_cfg.backend, E_exact, num_layers=1)
E_VQE = engine.energy(H, build_hva_layers(x_opt, backend=run_cfg.backend, num_layers=1))
print("VQE done. Beispiel-Energie:", E_VQE)
print("Exakte Energie:", E_exact, "Rel. Fehler:", abs(E_exact - E_VQE)/abs(E_exact)*100, "%")


## Subspace

In [ ]:
from src.subspace import unique_paulis_up_to_N, generate_all_transformed_hamiltonians, generate_all_pauli_products
import numpy as np
from scipy.linalg import fractional_matrix_power

# Optimierten Circuit aus VQE:
circ_opt = build_hva_layers(x_opt, backend=run_cfg.backend, num_layers=1)

# FGKS/Power-Basis bis N
pauli_strings = [p.to_label() for p in H.paulis]
basis = list(unique_paulis_up_to_N(pauli_strings, N=1))
n = len(basis)

# Observables erzeugen
H_obs = generate_all_transformed_hamiltonians(H, basis)   # (P_i, P_k) -> SparsePauliOp
S_obs = generate_all_pauli_products(basis)                # (P_i, P_k) -> SparsePauliOp

# Subraum-Matrizen messen
H_red = np.zeros((n, n), dtype=complex)
S_red = np.zeros((n, n), dtype=complex)

for (Pi, Pk), Obs in H_obs.items():
    i, j = basis.index(Pi), basis.index(Pk)
    H_red[i, j] = engine.expectations_for_observable(Obs, circ_opt)
    S_red[i, j] = engine.expectations_for_observable(S_obs[(Pi, Pk)], circ_opt)


H_red = 0.5*(H_red + H_red.conj().T); H_red = np.real_if_close(H_red)
S_red = 0.5*(S_red + S_red.conj().T); S_red = np.real_if_close(S_red)

# Regularisierung & EVP (wie bei dir)
w, U = np.linalg.eigh(np.real(S_red))
mask = w > 1e-2
V = U[:, mask]
S_proj = V.T.conj() @ S_red @ V
H_proj = V.T.conj() @ H_red @ V

S_inv_sqrt = fractional_matrix_power(S_proj, -0.5)
H_eff = S_inv_sqrt @ H_proj @ S_inv_sqrt

evals, _ = np.linalg.eigh(H_eff)
E_qse = np.min(evals)

E_exact = compute_exact_ground_energy(H)
print("Subspace-VQE Energie:", E_qse, "Rel. Fehler:", abs(E_exact - E_qse)/abs(E_exact)*100, "%")


## Everything in one

In [ ]:
from src.wrapper import run_experiment

res = run_experiment(hx=0.1, hz=2.0, J=-1,
                     num_layers=1,
                     mode="qasm_fake",   # "statevector" | "qasm_ideal" | "qasm_fake"
                     subspace=True,
                     shots=8192,
                     N_subspace=1)


print("Mode:", res.mode.name, "Subspace:", res.subspace_used)
print("VQE   E =", res.vqe_energy)
print("QSE   E =", res.subspace_energy)
print("Verbesserung ggü. VQE [%] =", res.improvement_vs_vqe_pct)
print("Rel. Fehler VQE   =", res.rel_error_vqe*100, "%")
print("Rel. Fehler QSE   =", res.rel_error_subspace*100, "%")
print("Meta:", res.meta)


In [ ]:
from src.modes import RunConfig, EstimatorMode
from qiskit_ibm_runtime.fake_provider import FakeQuitoV2
import numpy as np

# Beispiel: statevector
run_cfg = RunConfig(mode=EstimatorMode.STATEVECTOR, backend=FakeQuitoV2())

# Beispiel: qasm_ideal
# run_cfg = RunConfig(mode=EstimatorMode.QASM_IDEAL, backend=FakeQuitoV2())

# Beispiel: qasm_fake
# run_cfg = RunConfig(mode=EstimatorMode.QASM_FAKE, backend=FakeQuitoV2(), shots=8192)

from src.optimize import run_hva_grid_scan

error_grid, param_grid = run_hva_grid_scan(
    run_cfg,
    hx_range=np.linspace(-2, 2, 20),
    hz_range=np.linspace(-2, 2, 20),
    J=-1,
    num_layers=1,
    plot_2d=True,
    plot_3d=False
)

In [ ]:
from src.wrapper import run_experiment
from qiskit_ibm_runtime.fake_provider import FakeQuitoV2

res = run_experiment(hx=0.1, hz=2.0, J=-1,
                     num_layers=1,
                     mode="qasm_fake",
                     subspace="fine", 
                     shots=2048,
                     N_subspace=1,
                     backend=FakeQuitoV2())

print("Mode:", res.mode.name, "Subspace:", res.subspace_mode)
print("VQE   E =", res.vqe_energy)
print("QSE   E =", res.subspace_energy)
print("Verbesserung ggü. VQE [%] =", res.improvement_vs_vqe_pct)
print("Rel. Fehler VQE   =", res.rel_error_vqe*100, "%")
print("Rel. Fehler QSE   =", res.rel_error_subspace*100, "%")
print("Meta:", res.meta)